<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/thermodynamics/Phase_envelopes_of_oil_and_gas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Phase enevelopes of oil and gas
#@markdown Demonstration of phase envelopes of gas and oil. This notebook installs NeqSim and imports the nessesary packages, and then runs calculations in Python/Colab.
#@markdown <br><br>This document is part of the module ["Introduction to Gas Processing using NeqSim in Colab"](https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/examples_of_NeqSim_in_Colab.ipynb#scrollTo=_eRtkQnHpL70).
%%capture
!pip install neqsim
import neqsim
from neqsim.thermo.thermoTools import *
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

## Reservoir types
A multicomponent mixture exhibits an envelope for liquid/vapor phase change in the pressure/temperature diagram, which contains a bubble-point line and a dew-point line, compared with only a phase change line for a pure component. The various reservoir types of oil and gas systems based on the phase behavior of hydrocarbons in the reservoir, in

which the following five types of reservoirs are distinguished:

Typical Phase Diagram of Hydrocarbons
* Black oils;
* Volatile oils;
* Condensate (retrograde gases);
* Wet gases;
* Dry gases.

The amount of heavier molecules in the hydrocarbon mixtures varies from large to small in the black oils to the dry gases, respectively.

In [ ]:
#@title Introduction to Phase Envelopes
#@markdown This video gives an intriduction to behavour phase envelopes and phase changes and why this is Important
from IPython.display import YouTubeVideo
YouTubeVideo('OUS-ER3eemI', width=600, height=400)

#Calculation of phase behaviour of single components
A single component boils at constant temperature for a given pressure. This is represented by the boiling point line in a PT-diagram.

In the form below we calculate the vapour pressure curve of a single component from the triple point temperature up to the critical point. SRK of PR-EoS can be used as basis for the calculations.

In [ ]:
eosname = 'srk' #@param ["srk", "pr"]
camponentName = "methane" #@param ["CO2", "nitrogen", "methane", "ethane", "propane", "i-butane", "n-butane","n-pentane","n-hexane","n-heptane","n-octane"]
fluid1 = fluid('eosname') #create a fluid using the SRK-EoS
fluid1.addComponent(camponentName, 1.0) #adding 1 mole methane to the fluid

TTrip = fluid1.getPhase(0).getComponent(camponentName).getTriplePointTemperature()
PTrip = fluid1.getPhase(0).getComponent(camponentName).getTriplePointPressure()
Tcritical = fluid1.getPhase(0).getComponent(camponentName).getTC()
Pcritical = fluid1.getPhase(0).getComponent(camponentName).getPC()

fluid1.setTemperature(TTrip)
fluid1.setPressure(PTrip)
print('triple point temperature ', TTrip, "[K] and pressure ", PTrip, "[bara]")
print('critical temperature ', Tcritical, "[K] and pressure ", Pcritical, "[bara]")

def bubleP(pressure):
    fluid1.setPressure(pressure)
    bubt(fluid1)
    return fluid1.getTemperature('C')

pressure = np.arange(PTrip, Pcritical-5.0, 1.0)
temperature = [bubleP(P) for P in pressure]

plt.plot(temperature, pressure);
plt.xlabel('Temperature [C]');
plt.ylabel('Pressure [bara]');

#Calculation of Px diagram of binary mixtures
Calculation of VLE phase diagrams of binary mixtures. In a P-x diagram, the bubble point and dew point curves bound the two-phase region at its top and its bottom, respectively. The single-phase liquid region is found at high pressures; the single-phase vapor region is found at low pressures. In the T-x diagram (Figure 5.4), this happens in the reverse order; vapor is found at high temperatures and liquid at low temperatures. Consequently, the bubble point and dew point curve are found at the bottom and the top of the two-phase region, respectively.


In the form below you can calculate the PX and TX diagram of a binary fluid. Select two components, the pressure of the TX diagram and the temperature of  the PX diagram.

In [ ]:
bubtemp = []
dewtemp = []

bubpres = []
dewpres = []

x = []
y = []
x2 = []
y2 = []
comp1 = "propane" #@param ["methane", "ethane", "propane", "i-butane", "n-butane", "i-pentane", "n-pentane","n-hexane"]
comp2 = "n-hexane" #@param ["methane", "ethane", "propane", "i-butane", "n-butane", "i-pentane", "n-pentane","n-hexane"]
TXpressure = 11.0  #@param {type:"number"}
PXtemperature = 50.0  #@param {type:"number"}

Tcritical = fluid1.getPhase(0).getComponent(comp2).getTC()
Pcritical = fluid1.getPhase(0).getComponent(comp2).getPC()


fluid1 = fluid('srk')
addComponent(fluid1, comp1,0.000001)
addComponent(fluid1, comp2,1.01)
fluid1.setMixingRule('classic')
fluid1.setTemperature(Tcritical/2.0)
fluid1.setPressure(TXpressure)

fluid2 = fluid('srk')
addComponent(fluid2, comp1,0.000001)
addComponent(fluid2, comp2,1.01)
fluid2.setMixingRule('classic')
fluid2.setTemperature(PXtemperature+273.15)
fluid2.setPressure(Pcritical/20.0)

try:
    for temp in range(0,11):
        bubtemp.append(bubt(fluid1)-273.15)
        dewtemp.append(dewt(fluid1)-273.15)
        fluid1.init(0)
        x.append(fluid1.getPhase(0).getComponent(0).getz())
        fluid1.addComponent(comp2, -0.1)
        fluid1.addComponent(comp1, 0.1)
except:
        print('no dewt found')
try:
    for temp in range(0,11):
      # bubpres.append(bubp(fluid2))
        dewpres.append(dewp(fluid2))
        fluid2.init(0)
        fluid2.addComponent(comp2, -0.1)
        fluid2.addComponent(comp1, 0.1)
except:
        print('no dewt found')
        
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(x, bubtemp, label='bubT')
plt.plot(x, dewtemp, label="dewT")
plt.xlabel('x [mol fraction comp 1]')
plt.ylabel('Temperature [C]')

plt.subplot(2, 1, 2)
#plt.plot(x, bubpres, label='bubP')
plt.plot(x, dewpres, label="dewP")
plt.xlabel('x [mol fraction comp 1]')
plt.ylabel('Pressure [bara]')

#Calculation of phase envelope of a lean gas reservoir
Phase Envelope (two-phase region): The region enclosed by the bubble-point curve and the dew point curve wherein gas and liquid coexist in equlibrium, is identified as the phase envelope of the hydrocarbon system. A gas reservoir is dominated by light components, and generally contain low amounts of C7+ components. 

In the form below you can specify the molar composition of a lean natural gas (heaviest component is n-hexane). The composition will be normalizes to make sure sum of molefractions is 1.

In [ ]:
nitrogen = 1.0 #@param {type:"number"}
CO2 = 2.5 #@param {type:"number"}
methane = 80.0  #@param {type:"number"}
ethane = 5.0  #@param {type:"number"}
propane =  2.5 #@param {type:"number"}
ibutane =  1.25 #@param {type:"number"}
nbutane =  1.25 #@param {type:"number"}
ipentane =  0.5 #@param {type:"number"}
npentane =  0.5 #@param {type:"number"}
nhexane =  0.05 #@param {type:"number"}

fluid1 = fluid('srk')
fluid1.addComponent("nitrogen", nitrogen)
fluid1.addComponent("CO2", CO2)
fluid1.addComponent("methane", methane)
fluid1.addComponent("ethane", ethane)
fluid1.addComponent("propane", propane)
fluid1.addComponent("i-butane", ibutane)
fluid1.addComponent("n-butane", nbutane)
fluid1.addComponent("i-butane", ibutane)
fluid1.addComponent("n-pentane", nbutane)
fluid1.addComponent("n-pentane", nbutane)
fluid1.addComponent("n-hexane", nbutane)
fluid1.setMixingRule('classic');

thermoOps = neqsim.thermodynamicOperations.ThermodynamicOperations(fluid1)
thermoOps.calcPTphaseEnvelope()

plt.plot(list(thermoOps.getOperation().get("dewT")),list(thermoOps.getOperation().get("dewP")), label="dew point")
plt.plot(list(thermoOps.getOperation().get("bubT")),list(thermoOps.getOperation().get("bubP")), label="bubble point")
plt.title('PT envelope')
plt.xlabel('Temperature [\u00B0C]')
plt.ylabel('Pressure [bar]')
plt.legend()
plt.show()

#Calculation of phase envelope of a gas-condensate reservoir
A gas-condesate reservoir has more   heavy components, and generally contain hogh amounts of C7+ components.

In the form below you can specify the molar composition of a gas-condesate reservoir fluid (heaviest component is C7+). The composition will be normalizes to make sure sum of molefractions is 1.

In [ ]:
nitrogen = 0.51 #@param {type:"number"}
CO2 = 2.51 #@param {type:"number"}
methane = 20.0  #@param {type:"number"}
ethane = 4.01  #@param {type:"number"}
propane =  3.01 #@param {type:"number"}
ibutane =  1.25 #@param {type:"number"}
nbutane =  1.25 #@param {type:"number"}
ipentane =  0.25 #@param {type:"number"}
npentane = 0.25 #@param {type:"number"}
nhexane =  0.125 #@param {type:"number"}
C7 =  4.05 #@param {type:"number"}
C7Molarmass =  210.0 #@param {type:"number"}
C7Density =  810.2 #@param {type:"number"}

fluid1 = fluid('srk')
fluid1.addComponent("nitrogen", nitrogen)
fluid1.addComponent("CO2", CO2)
fluid1.addComponent("methane", methane)
fluid1.addComponent("ethane", ethane)
fluid1.addComponent("propane", propane)
fluid1.addComponent("i-butane", ibutane)
fluid1.addComponent("n-butane", nbutane)
fluid1.addComponent("i-pentane", ipentane)
fluid1.addComponent("n-pentane", npentane)
fluid1.addComponent("n-hexane", nhexane)
fluid1.addPlusFraction("C7", C7, C7Molarmass/1000.0, C7Density/1000.0)
fluid1.getCharacterization().characterisePlusFraction()
fluid1.setMixingRule('classic')

phaseenvelope(fluid1, True)

# Calculation of the phase envelope of an export gas
In the following example we calculate the phase envelope of a typical export gas. The cricondentherm and cricondebar will be reported.

In [ ]:
import pandas as pd

naturalgas = {'ComponentName':  ["nitrogen", "CO2", "methane", "ethane", "propane", "i-butane", "n-butane","i-pentane", "n-pentane","2-m-C5", "3-m-C5", "n-hexane", "benzene", "c-hexane", "n-heptane", "toluene", "c-C7", "n-octane", "m-Xylene", "c-C8", "n-nonane", "nC10"], 
       'MolarComposition[-]':  [1.1799,0.5102,95.3303,2.1102,0.3217,0.1278,0.0846,0.0694,0.0340,0.0335,0.0109,0.0181,0.0017,0.0661,0.0207,0.0045,0.0530,0.0061,0.0033,0.000103,0.0032,0.0004]
} 


naturalgasdf = pd.DataFrame(naturalgas) 
print("Natural Gas Fluid:\n")
print(naturalgasdf.head(30).to_string())

naturalgasFluid = fluid_df(naturalgasdf)

gasPhaseEnvelope = phaseenvelope(naturalgasFluid,True)
cricobar = gasPhaseEnvelope.get("cricondenbar")
cricotherm = gasPhaseEnvelope.get("cricondentherm")
print("cricobarP ",  cricobar[1], "  [bara] ", " cricobarT ", cricobar[0], " °C")
print("cricothermP ",  cricotherm[1], "  [bara] ", " cricothermT ", cricotherm[0], " °C")